## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [10]:
from sqlalchemy import create_engine, text, MetaData, select, desc, func
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [2]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3.
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [3]:
### Imprima os três primeiros registros da tabela tracks
tracks = select(Base.classes.tracks).limit(3)
tracks_res = session.scalars(tracks).all()

for track in tracks_res:
    print(track.TrackId, track.Name, track.AlbumId, track.MediaTypeId, track.GenreId, track.Composer, track.Milliseconds, track.Bytes, track.UnitPrice)

1 For Those About To Rock (We Salute You) 1 1 1 Angus Young, Malcolm Young, Brian Johnson 343719 11170334 0.99
2 Balls to the Wall 2 2 1 None 342562 5510424 0.99
3 Fast As a Shark 3 2 1 F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman 230619 3990994 0.99


In [4]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
Track = Base.classes.tracks
Album = Base.classes.albums

query = (
    select(Track.Name, Album.Title)
    .join(Album, Track.AlbumId == Album.AlbumId)
    .limit(20)
)

result = session.execute(query).all()

for track_name, album_title in result:
    print(f"Faixa: {track_name}, Álbum: {album_title}")



Faixa: For Those About To Rock (We Salute You), Álbum: For Those About To Rock We Salute You
Faixa: Put The Finger On You, Álbum: For Those About To Rock We Salute You
Faixa: Let's Get It Up, Álbum: For Those About To Rock We Salute You
Faixa: Inject The Venom, Álbum: For Those About To Rock We Salute You
Faixa: Snowballed, Álbum: For Those About To Rock We Salute You
Faixa: Evil Walks, Álbum: For Those About To Rock We Salute You
Faixa: C.O.D., Álbum: For Those About To Rock We Salute You
Faixa: Breaking The Rules, Álbum: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives, Álbum: For Those About To Rock We Salute You
Faixa: Spellbound, Álbum: For Those About To Rock We Salute You
Faixa: Balls to the Wall, Álbum: Balls to the Wall
Faixa: Fast As a Shark, Álbum: Restless and Wild
Faixa: Restless and Wild, Álbum: Restless and Wild
Faixa: Princess of the Dawn, Álbum: Restless and Wild
Faixa: Go Down, Álbum: Let There Be Rock
Faixa: Dog Eat Dog, Álbum: Let There Be Rock


In [6]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
# from sqlalchemy import func

# max_resul= session.execute(select(func.max(Base.classes.invoice_items.Quantity))).scalar()
# print(max_resul)

query = (
    select(Base.classes.invoice_items.Quantity, Base.classes.tracks.Name)
    .join(Base.classes.tracks, Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .order_by(Base.classes.invoice_items.Quantity)
    .limit(10)
)

results = session.execute(query).all()

for quantity, track_name in results:
    print(f"Quantidade: {quantity}, Faixa: {track_name}")



1
Quantidade: 1, Faixa: Balls to the Wall
Quantidade: 1, Faixa: Restless and Wild
Quantidade: 1, Faixa: Put The Finger On You
Quantidade: 1, Faixa: Inject The Venom
Quantidade: 1, Faixa: Evil Walks
Quantidade: 1, Faixa: Breaking The Rules
Quantidade: 1, Faixa: Dog Eat Dog
Quantidade: 1, Faixa: Overdose
Quantidade: 1, Faixa: Love In An Elevator
Quantidade: 1, Faixa: Janie's Got A Gun


In [11]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
query_top_selling_tracks = (
    select(Base.classes.tracks.Name, func.count(Base.classes.invoice_items.TrackId).label("total_sales"))
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.tracks.TrackId)
    .order_by(desc("total_sales"))
    .limit(10)
)

top_selling_tracks_result = session.execute(query_top_selling_tracks).all()

for track_name, total_sales in top_selling_tracks_result:
    print(f"Faixa: {track_name}, Vendas: {total_sales}")

Faixa: Balls to the Wall, Vendas: 2
Faixa: Inject The Venom, Vendas: 2
Faixa: Snowballed, Vendas: 2
Faixa: Overdose, Vendas: 2
Faixa: Deuces Are Wild, Vendas: 2
Faixa: Not The Doctor, Vendas: 2
Faixa: Por Causa De Você, Vendas: 2
Faixa: Welcome Home (Sanitarium), Vendas: 2
Faixa: Snowblind, Vendas: 2
Faixa: Cornucopia, Vendas: 2


In [12]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

query_top_selling_artists = (
    select(Base.classes.artists.Name, func.count(Base.classes.invoice_items.InvoiceLineId).label("total_sales"))
    .join(Base.classes.albums, Base.classes.artists.ArtistId == Base.classes.albums.ArtistId)
    .join(Base.classes.tracks, Base.classes.albums.AlbumId == Base.classes.tracks.AlbumId)
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.artists.Name)
    .order_by(desc("total_sales"))
    .limit(10)
)

top_selling_artists_result = session.execute(query_top_selling_artists).all()

for artist_name, total_sales in top_selling_artists_result:
    print(f"Artista: {artist_name}, Vendas: {total_sales}")

Artista: Iron Maiden, Vendas: 140
Artista: U2, Vendas: 107
Artista: Metallica, Vendas: 91
Artista: Led Zeppelin, Vendas: 87
Artista: Os Paralamas Do Sucesso, Vendas: 45
Artista: Deep Purple, Vendas: 44
Artista: Faith No More, Vendas: 42
Artista: Lost, Vendas: 41
Artista: Eric Clapton, Vendas: 40
Artista: R.E.M., Vendas: 39
